In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sal = pd.read_csv('salaries.csv')
jon = pd.read_csv('US_ZIP_mean_median_income.csv')
ian = pd.read_csv('Zip_MedianValuePerSqft_AllHomes.csv')
ian2 = pd.read_csv('Zip_Zhvi_Summary_AllHomes.csv')
andrew = pd.read_csv('zip_code_data.csv')




In [3]:
andrew.rename(columns={'B19013_001': 'MedianIncome'})
andrew.rename(columns={'Unnamed: 0': 'Zip'})
andrew['Zip'] = andrew['Unnamed: 0'].str[6:]
print(andrew.head(5))

    Unnamed: 0  B19013_001  no_male_hs_p  no_female_hs_p  at_least_hs_male_p  \
0  ZCTA5 01001       59453      6.905465        8.535844           93.094535   
1  ZCTA5 01002       54395      3.913674        3.683004           96.086326   
2  ZCTA5 01003         NaN      0.000000        0.000000          100.000000   
3  ZCTA5 01005       74167      8.004709        7.709373           91.995291   
4  ZCTA5 01007       75502      4.325260        7.790476           95.674740   

   at_least_hs_female_p  at_least_bach_male_p  at_least_bach_female_p  \
0             91.464156             25.683161               22.357230   
1             96.316996             71.718473               70.378332   
2            100.000000             35.467980               76.470588   
3             92.290627             29.723367               34.775374   
4             92.209524             39.121972               40.742857   

   male_unemployment  female_unemployment  ...    \
0           5.574913        

In [14]:


sal['Zip'] = sal['Zip'].str[0:5]

sal.tail(5)

,Function,sup1,sup2,sup3,sup4,sup5,salary,gender,disabled,race,...,inst4,inst5,instBudget,City,State,Zip,country,Category,instSize,instType
2304,position8,0,0,0,9,0,45000,Woman,False,Caucasian,...,9,4,480000,Weyers Cave,VA,24486,United States,two,NaN,NaN
2305,position9,0,0,0,9,0,20000,Woman,False,Caucasian,...,9,4,480000,Weyers Cave,VA,24486,United States,two,NaN,NaN
2306,position6,0,0,0,9,0,22000,Woman,False,Caucasian,...,9,4,480000,Weyers Cave,VA,24486,United States,two,NaN,NaN
2307,position25,0,0,0,9,0,20000,Woman,False,Caucasian,...,9,4,480000,Weyers Cave,VA,24486,United States,two,NaN,NaN
2308,position6,0,0,0,9,0,20000,Woman,False,Caucasian,...,9,4,480000,Weyers Cave,VA,24486,United States,two,NaN,NaN


In [15]:
sal.mean()
result = pd.merge(sal, andrew, how = 'left',  on='Zip')
#result2 = pd.concat([result3, ian], axis=1)
#result1 = pd.concat([result2, ian2], axis=1)
#result = pd.concat([result1, andrew], axis=1)

result.head(5)

result.to_csv('merged.csv')

In [ ]:
# eliminate rows with missing Y values (NaN)
sal['missingSalary'] = pd.isnull(sal['salary'])
sal2 = sal[(sal.missingSalary == False)]

In [ ]:
# list of X vars to include
X_numeric_features = ['sup1', 'sup2', 'sup3', 'sup4', 'sup5', 'disabled', 'yearsInPosition', 
         'yearsInProfession', 'age', 'cred1', 'cred2', 'inst1', 'inst2', 'inst3', 'inst4', 'inst5', 'instBudget', 'instSize']
X_numeric = sal2[X_numeric_features]
X_categorical_features = ['Function', 'gender', 'race', 'highestDegree', 'Category', 'instType']
X_categorical = sal2[X_categorical_features]

In [ ]:
# create dummy variables for each of the categorical features
# DOC: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
#converts dummies to 1s and 0s 

Function_dummies = pd.get_dummies(X_categorical['Function'])
gender_dummies = pd.get_dummies(X_categorical['gender'])
race_dummies = pd.get_dummies(X_categorical['race'])
highestDegree_dummies = pd.get_dummies(X_categorical['highestDegree'])
Category_dummies = pd.get_dummies(X_categorical['Category'])
instType_dummies = pd.get_dummies(X_categorical['instType'])

X_dummy_features = pd.concat([Function_dummies, gender_dummies, race_dummies, highestDegree_dummies, Category_dummies, instType_dummies], axis=1)

# convert to ndarray
X_dummy_features = X_dummy_features.as_matrix()

In [ ]:
# impute missing values in numerical features
# DOC: http://scikit-learn.org/stable/modules/preprocessing.html

from sklearn.preprocessing import Imputer
imp = Imputer()
imp.fit(X_numeric)
X_numeric_imputed = imp.transform(X_numeric)



In [ ]:
X = np.concatenate((X_dummy_features, X_numeric_imputed), axis=1)

In [ ]:
# y is salary
y = sal2.iloc[:, 6].values

In [ ]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [ ]:
# keep track of variance on test data, to graph
var_to_graph = {}
# bring residual sum of squares from regression1.ipynb
var_to_graph['simpReg'] = 265376883.08

In [ ]:
from sklearn import datasets, linear_model
# DOC: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
X_train_no_intercept = X_train
X_train = X_train.reshape(-1, X_train.shape[1])
regr.fit(X_train, y_train)

# The intercept
print('Intercept: \n', regr.intercept_)
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean square error
print("Residual sum of squares, training data: %.2f"
      % np.mean((regr.predict(X_train) - y_train) ** 2))
print("Residual sum of squares, test data: %.2f"
      % np.mean((regr.predict(X_test) - y_test) ** 2))
var_to_graph['multReg_linear'] = np.mean((regr.predict(X_test) - y_test) ** 2)
# Explained variance score: 1 is perfect prediction
print('Variance score, training data: %.2f' % regr.score(X_train, y_train))
#vector of prediction error
print('Distribution of prediction error on training data:')
predError = regr.predict(X_train) - y_train
plt.hist(predError)
plt.show()

print('Distribution of prediction error on test data:')
predError = regr.predict(X_test) - y_test
plt.hist(predError)
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# DOC: http://scikit-learn.org/stable/modules/preprocessing.html
#taking same features and applying different powers to them 

poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X)

In [ ]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X_poly, y, test_size=0.3, random_state=0)

In [ ]:
## POLYNOMINAL 
# Create linear regression object
poly = linear_model.LinearRegression(normalize=True)

# Train the model using the training sets
X_train_no_intercept = X_train
X_train = X_train.reshape(-1, X_train.shape[1])
poly.fit(X_train, y_train)

# The intercept
print('Intercept: \n', poly.intercept_)
# The coefficients
print('Coefficients: \n', poly.coef_)
# The mean square error
print("Residual sum of squares, training data: %.2f"
      % np.mean((poly.predict(X_train) - y_train) ** 2))
print("Residual sum of squares, test data: %.2f"
      % np.mean((poly.predict(X_test) - y_test) ** 2))
var_to_graph['multReg_poly'] = np.mean((poly.predict(X_test) - y_test) ** 2)
# Explained variance score: 1 is perfect prediction
print('Variance score, training data: %.2f' % poly.score(X_train, y_train))
#vector of prediction error
print('Distribution of prediction error on training data:')
predError = poly.predict(X_train) - y_train
plt.hist(predError)
plt.show()

print('Distribution of prediction error on test data:')
predError = poly.predict(X_test) - y_test
plt.hist(predError)
plt.show()

In [ ]:
## RIDGE REGRESSION
# DOC: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

# Create linear regression object
regr = linear_model.Ridge()

# Train the model using the training sets
X_train_no_intercept = X_train
X_train = X_train.reshape(-1, X_train.shape[1])
regr.fit(X_train, y_train)

# The intercept
print('Intercept: \n', regr.intercept_)
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean square error
print("Residual sum of squares, training data: %.2f"
      % np.mean((regr.predict(X_train) - y_train) ** 2))
print("Residual sum of squares, test data: %.2f"
      % np.mean((regr.predict(X_test) - y_test) ** 2))
var_to_graph['multReg_ridge'] = np.mean((regr.predict(X_test) - y_test) ** 2)
# Explained variance score: 1 is perfect prediction
print('Variance score, training data: %.2f' % regr.score(X_train, y_train))
#vector of prediction error
print('Distribution of prediction error on training data:')
predError = regr.predict(X_train) - y_train
plt.hist(predError)
plt.show()

print('Distribution of prediction error on test data:')
predError = regr.predict(X_test) - y_test
plt.hist(predError)
plt.show()

In [ ]:
# bar graph of dict with Residual Sum of Squares on test datasets

var_to_graph['multReg_poly'] = 0
#SET ABOVE TO ZERO TO REMOVE MULREG_POLY BECAUSE IT WAS TOO CRAZY DUE TO OUTLIERS AND OVERFITTING
plt.bar(range(len(var_to_graph)), var_to_graph.values(), align='center')
plt.xticks(range(len(var_to_graph)), var_to_graph.keys())

plt.show()